# Clean Extraction Method

This notebook is just a cleaner version of "extraction_data.ipynb", its objective is to provide a clearer methodology of data generation. Though, it is not certain that it will cover all the materials that "extraction_data.ipynb" covered. Then, take it as a complement for the ease of the lector rather than a subsitute for the developper.

## Sample_2023_with_txt

This how we generated the data set sample_2023_woth_txt

First we generate the file with the sample of interest

In [ ]:
import pandas as pd

def import_sample(file, format:str, begin=None, end=None):
    if format == 'excel':
        df = pd.read_excel(file).iloc[begin:end]
        print(df.info())
        return df
    elif format == 'csv':
        df = pd.read_csv(file).iloc[begin:end]
        print(df.info())
        return df
    else:
        print('Please provide a valid format (excel or csv)')

df_samp = import_sample(r'C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\training_xlsx\sample2023.xlsx', 'excel', 0, 124)

In [ ]:
df_samp.columns
df_samp_final = df_samp[[
    'UrlLegifrance', 
    'Entreprise', 
    'Siret', 
    'Secteur', 
    # 'Nature', 
    # 'Titre',
    'Naf732', 
    'Date Texte', 
    # 'Date Maj', 
    # 'Date Dépot', 
    # 'Date diffusion',
    # 'Date fin', 
    # 'LesSyndicats', 
    # 'LesThemes', 
    # 'type',
    'Tranche Effectif(Base siren)', 
    'Fichier', 
    # 'ID'
    ]].copy()
df_samp_final.info()

In [ ]:
# Add a column for the application year
df_samp_final["Année d'application NAO"] = 2023
df_samp_final["code"] = df_samp_final["Fichier"].str.slice(101,113)
df_samp_final.head(10)

# Reorder the columns
df_samp_final.columns
df_samp_final = df_samp_final[[
    # 'UrlLegifrance',
    'code',
    'Entreprise', 
    'Siret', 
    'Secteur', 
    'Naf732', 
    'Date Texte',
    "Année d'application NAO",
    'Tranche Effectif(Base siren)', 
    ]].copy()

# Rename the columns
df_samp_final.columns = ["code","Nom de l'entreprise", "Siret", "Sous secteur Secafi", "Code NAF", "Date de l'accord du texte", "Année d'application NAO", "Tranche Effectif (Base siren)"]

df_samp_final.head(10)
df_samp_final.to_excel(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\training_xlsx\sample_2023_no_txt.xlsx", index=False)

Second we create the file with the corresponding code txt

In [ ]:
import pandas as pd
import numpy as np

# Read in the data
df_sample = pd.read_excel(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\training_xlsx\sample2023.xlsx") # this sample has been directly extracted from Qlik Sense plateform

# We select the columns we want to keep
df_sample = df_sample[[
    'UrlLegifrance', 
    # 'Entreprise', 
    # 'Siret', 
    # 'Secteur', 
    # 'Nature', 
    # 'Titre',
    # 'Naf732', 
    # 'Date Texte', 
    # 'Date Maj', 
    # 'Date Dépot', 
    # 'Date diffusion',
    # 'Date fin', 
    # 'LesSyndicats', 
    # 'LesThemes', 
    # 'type',
    # 'Tranche Effectif(Base siren)', 
    'Fichier', 
    'ID'
    ]].copy()

# We create a new column with the ID shortened and the column reordered
df_sample["code"] = df_sample["Fichier"].str.slice(101,113)
df_sample = df_sample[['UrlLegifrance', 'code','Fichier']].copy()
df_sample.head()

Our objective now is to extract the texts stored on our local data center.

In [ ]:
# We create a list of the unique ID_b and write in a doc file

def write_column_to_txt(df, column, path):
    """
    Write the content of a column of a dataframe in a txt file

    Args:
        df (dataframe): the dataframe
        column (str): the column to write
        path (str): the path of the txt file
    """
    with open(path, 'w') as f:
        for item in df[column].unique():
            f.write("%s\n" % item)

write_column_to_txt(df_sample, "Fichier", r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\docx\2022_2023\sample_2023\Liste.txt")

We download each of the txt files, we have to do this manually, since the program doesn't provide me access

Now we convert each of this text to the txt format, for that we can use the two following scripts : convert_to_txt_V2.py and clean_txt.py

In [ ]:
import subprocess
import os

# Specify the path to the folder containing the Python scripts
script_folder = r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\script"

# Define the names of the Python scripts you want to run
script1_name = "convert_to_txt_V2.py"
script2_name = "clean_txt.py"

# Build the full paths to the script files
script1_path = os.path.join(script_folder, script1_name)
script2_path = os.path.join(script_folder, script2_name)

# Run the first script
subprocess.run(["python", script1_path])

# Run the second script
subprocess.run(["python", script2_path])

Now we can do a data set out of the document and match their ID to the one of our sample

In [ ]:
import glob
import pandas as pd
import os
from tqdm.notebook import tqdm

directory = glob.glob(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\docx\2017_2021\Subset_1\text\*.txt")

# iniate an empty dataframe

final_df = pd.DataFrame()

for file in tqdm(directory):
    # collect the file name and store it in a list
    file_name = os.path.basename(file)[:-19]
    
    # collect the file content and store it in a list
    collected_text = []
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read()
        collected_text.append(text)

    df = pd.DataFrame(collected_text , columns=['text'])
    df.insert(0, 'code', file_name)
    final_df = pd.concat([final_df, df], ignore_index=True)

final_df.to_excel(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\training_xlsx\sample_2017_2021_sub1.xlsx", index=False)

## Extraction and matching of text for a sample

We build on what we did for the NAO 2023 to extract and matcht the text with their respective contract

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\training_xlsx\sample_2023_no_txt.xlsx")
df.head()

We match it with our text data set

In [ ]:
df_text = pd.read_excel(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\training_xlsx\sample2023_txt.xlsx")
df_text

In [ ]:
df_merged = pd.merge(df, df_text, on="code", how='left')

#Cleaning the dataframe
df_merged.dropna(subset=['text'], inplace=True)
df_merged['text'] = df_merged['text'].str.replace(r'^\s+', '', regex=True)
df_merged['text'] = df_merged['text'].str.replace(r'\s+$', '', regex=True)

# Reduce the data set to two three columns
df_merged = df_merged[[
    "Nom de l'entreprise", 
    'code', 
    # 'Siret', 
    # 'Sous secteur Secafi',
    # 'Code NAF', 
    # "Date de l'accord du texte", 
    # "Année d'application NAO",
    # 'Tranche Effectif (Base siren)', 
    'text'
    ]].copy()

df_merged.head()

# Save the dataframe
df_merged.to_excel(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\training_xlsx\sample2023_with_txt.xlsx", index=False)

## Text and ID Dataset

In [ ]:
import glob
import pandas as pd
import os
from tqdm.notebook import tqdm

directory = glob.glob(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\docx\2022_2023\2022_2023_all\Subset_1\text\*.txt")

def cleaning_function(df):
    df.dropna(subset=['text'], inplace=True)
    df['text'] = df['text'].str.replace(r'^\s+', '', regex=True)
    df['text'] = df['text'].str.replace(r'\s+$', '', regex=True)
    return df

# iniate an empty dataframe

final_df = pd.DataFrame()

for file in tqdm(directory):
    # collect the file name and store it in a list
    file_name = os.path.basename(file)[:-19]
    
    # collect the file content and store it in a list
    collected_text = []
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read()
        collected_text.append(text)

    df = pd.DataFrame(collected_text , columns=['text'])
    df.insert(0, 'code', file_name)
    final_df = pd.concat([final_df, df], ignore_index=True)

final_df = cleaning_function(final_df)

In [ ]:
final_df["len"] = final_df["text"].agg(lambda x: len(x))
final_df.head()

In [ ]:
final_df.to_excel(r"C:\Users\garsonj\Desktop\spacy_finetuning\spacy_files\data\training_xlsx\sample_2022_2023_sub1.xlsx", index=False)